#  PIM_COLOR_EXTRACTION_111723.ipynb 
   - Module name: PIM_COLOR_EXTRACTION_111723.ipynb
   - Purpose: Apply rapidfuzz.process.extract match algorithm to get 'Detected color' to match color. 
   - The module includedthe following steps:
     - Data preparation
       1. Create a df 'pim_color' which contained 'TIM_ID, COLOR_CD_VLU' from 'PIM_Color_Attributes_111623.csv'.
       2. Read ITM_ID_GTIN_Map_111623.csv to create a df 'gtin_itm_id' which contained 'GTIN_NO, ITM_ID'.  
       3. created a column 'GTIN_NO' for 'pim_color' by mapping ITM_ID between 'pim_color' and  'gtin_itm_id' 
       4. Read 'PIM_DATA_W_TAG_DSC_110323.zip' to create a df 'pim_df'.
       5. Used 'GTIN_NO' as key to right join pim_color and pim_df to create a DF 'pim_load'. 
       6. In 'pim_load' renamed 'COLOR_CD_VLU' to 'COLOR',and appl
         fillna('NONE').
     - Match within DPT
       1. Created a DF colors_DPT with column name 'DPT_DSC' and 'UNIQUE COLORS' from pim_load by grouping 'DPT_DSC', and the 'UNIQUE COLORS'  is joining all the colors under the same DPT_DSC with '|' as a separator.
       2. Created a ndarray "color_DPT_DSCs" from 'colors_DPT' by joining colors in 'COLOR.unique' separated by '|' and exclude 'NONE','OTHER'.
       3. Created a DF colors_COM with column name 'COM_CD' and 'UNIQUE COLORS' from pim_load by grouping 'COM_CD', and the 'UNIQUE COLORS'  is joining all the colors under the same COM_CD with '|' as a separator.
       4. Created a ndarray "color_COM_CDs" from 'colors_COM' by joining colors in 'COLOR.unique' separated by '|' and exclude 'NONE','OTHER'.
       5. Excluded COMS without color from pim_load
       6. Created a column 'MATCHING_DESCRIPTION' for pim_load by concatenating 'ITM_LVL_DSC', 'VND_ECOM_DSC',and 'TAG_DSC'
       7. Ran a loop for all DPT from colors_DPT.DPT_DSC to invoke 'color_match' function to get 'Detected color'  by matching 'MATCHING_DESCRIPTION' with 'color_list' and keep the result in a excel worksheet.
   - Function "color_match" 
       - Purpose: Called/Invoked rapidfuzz.process.extract twice to match color.
       - "color_match" had the following parameters:
         - item: The string you want to find the color. It passed the value to ‘query’ in rapidfuzz.process.extract. It used  the value of pim_dpt.MATCHING_DESCRIPTION.
         - color_list_org: A list of all colors the item should be compared with. It passed the value to ‘choice’ in rapidfuzz.process.extract. 
           - The function invoked rapidfuzz.process.extract twice. The first call used “fuzz.token_set_ratio” as a scorer, and used “color_list" for the matching process. The limit for the first call was 100.
           - Created “final_rematch “ from the second call used “fuzz.partial_ratio” as a score and used match list from the first call as “choice” to match the “color. The limit for the second call was 5.
         - Cutoff: Assigned value to score_cutoff which would be used to set a minimum similarity for the string improve to the performance. The first call, the default value was 85. For the second call, it used the maximum score from the first call.  
       - Return "matched"
         - Invoked function “remove_substrings“ to eliminate item in another item from 'final_rematch' created from the second call and created 'final_rematch'. 
         - The 'final_rematch' will match every word in the item to determined the contents of matched.
 
   - Input & Output
     - input 
       -  PIM_Color_Attributes_111623.csv - Extracted from CURIDQ.pimmart.CLS_SPC_ATB. 
       -  ITM_ID_GTIN_Map_111623.csv - Extracted from CURIDQ.PIMMART.ITEM_VIEW. 
          - Will  be used to add ITEM_NO to PIM_color_Attributes_111623.csv. 
       -  PIM_DATA_W_TAG_DSC_110323.zip 
          - A new PIM file where we can use TAG_DSC in addition to VND_ECOM_DSC and ITM_LVL_DSC. 
          - Would merge with  PIM_color_Attributes_111623.csv.
   - output: COLOR_DPT_DSC_mmddyy.xlsx.
          - For each DPT in colors_DPT.DPT_DSC, keep 'Detected color' in the worksheet with 'DPT' as a sheet_name. 
   - Notes
     - Cloned from PIM_COLOR_EXTRACTION_111623.ipynb written by Shiladitya Chakraborty
     - There were 429 colors in color_COM_CDs and 95 DPT from colors_DPT
   - written by: Sophia Yue
   - date: 11.17.23 

In [1]:
import pandas as pd, numpy as np, re, time
from rapidfuzz import process, fuzz,utils
np.set_printoptions(threshold=np.inf)

pd.options.display.float_format = '{:20,.2f}'.format 

from datetime import date
from pandas import ExcelWriter
from pandas import ExcelFile



import datetime as dt
pd.options.display.float_format = '{:20,.2f}'.format 
pd.set_option("display.max_rows", None)
path = 'C:\\users\\iny2819\\Kroger\\Data\\' 
path_atb = path + 'ATB\\'
path_atb_yue = path + 'ATB\\atb_yue\\'

In [2]:
path_code = 'C:\\users\\iny2819\\kroger\\Code\\'  
f_com_code = path_code + "com_code.py"

exec(compile(open(f_com_code , "rb").read(), f_com_code, 'exec' ))

In [3]:
import warnings
from pandas.errors import SettingWithCopyWarning
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

In [4]:
#pim_color = pd.read_csv('PIM_Color_Attributes_102523.csv')
pim_color = pd.read_csv(path_atb + 'PIM_color_Attributes_111623.csv')
#gtin_itm_id = pd.read_csv('ITM_ID_GTIN_102523.csv')
gtin_itm_id = pd.read_csv(path + 'ITM_ID_GTIN_Map_111623.csv')
pim_color['GTIN_NO'] = pim_color.ITM_ID.map(dict(zip(gtin_itm_id.ITM_ID, gtin_itm_id.GTIN_NO)))
#flavors_COM = pd.DataFrame(pim_flavors.groupby('COM_CD').apply(lambda x: '|'.join(x.Flavor_Code.unique())), columns= ['UNIQUE FLAVORS']).reset_index()

In [5]:
pim_df = pd.read_csv(path_atb + 'PIM_DATA_W_TAG_DSC_110323.zip')

In [6]:
pim_load = pim_color.merge(pim_df, how = 'right', on = 'GTIN_NO')
# SCENT_VLU different from SCENT_CD
pim_load[pim_load.COLOR_VLU.notnull() & (pim_load.COLOR_VLU != pim_load.COLOR_CD)]\
.drop_duplicates(['COLOR_CD','COLOR_VLU'])[['COLOR_CD','COLOR_VLU']][0:10]

,COLOR_CD,COLOR_VLU
280,MULTI_COLORED,MULTI-COLORED
321,NaN,clear/white
377,NaN,Seafoam
456,NaN,Light/Medium
591,NaN,IRON GATE
654,NaN,Fair
804,LIGHT_PINK,LIGHT PINK
902,NaN,TOURMALINE
908,NaN,HAVANA
911,NaN,BLUE and TAN


In [7]:
# USE _VLU if present, else use _CD
#pim_load['COLOR'] = np.where(pim_load.COLOR_VLU.notnull(), pim_load.COLOR_VLU, pim_load.COLOR_CD)
pim_load['COLOR'] = pim_load.COLOR_CD_VLU
print(f'2. (pim_load.COLOR.dropna.unique {pim_load.COLOR.dropna().unique()}')

2. (pim_load.COLOR.dropna.unique ['RED' 'GREEN' 'BLACK' 'CLEAR' 'NONE' 'NAVY' 'BEIGE' 'BLUE' 'ORANGE'
 'MULTI_COLORED' 'SAGE' 'clear/white' 'OTHER' 'COPPER' 'IVORY' 'MOCHA'
 'Seafoam' 'PLUM' 'TAN' 'BROWN' 'Light/Medium' 'NATURAL' 'BRUNETTE'
 'BLONDE' 'PINK' 'IRON GATE' 'WHITE' 'GRAY' 'SILVER' 'Fair' 'PURPLE'
 'YELLOW' 'ROSE' 'LIGHT_PINK' 'BURGUNDY' 'PEACH' 'TOURMALINE' 'HAVANA'
 'BLUE and TAN' 'ALLOY' 'COCONUT' 'BROWN_BLACK' 'MEDIUM' 'HONEY' 'Cafe'
 'silver satin finish steel' 'CORAL' 'CAMEO' 'White' 'CARAMEL'
 'Silver lid' 'ZINC' 'CHOCOLATE' 'GOLD' 'black/silver' 'LAVENDER' 'TAUPE'
 'DEEP' 'LIGHT' 'TOFFEE' 'METALLIC_GRAY' 'APRICOT'
 'Multi color design on shaker' 'METALLIC_SILVER' 'DARK_RED_BROWN' 'MAUVE'
 'CHARCOAL_GREY' 'NUDE' 'RICH BROWN' 'EXTREME BLACK' 'SAPPHIRE' 'BRONZE'
 'Black' 'nude' 'FAIR' 'FUCHSIA' 'BRASS' 'LIGHT_BROWN' 'CHARCOAL' 'CHERRY'
 'Medium / Tan' 'LIGHT MEDIUM' 'DARK' 'FAIR LIGHT' 'Black handle' 'LILAC'
 'Teal' 'BABY_BLUE' 'CLAY' 'LIGHT_BLUE' 'Gray and Green' 'AMBE

In [8]:
pim_load['COLOR'] = pim_load.COLOR.fillna('NONE')

In [9]:
sorted(list(pim_load.COLOR.dropna().unique()))

['#1 NUDE',
 '0',
 '001',
 '005',
 '008',
 '00888068950327',
 '00888068954318',
 '010',
 '014',
 '047',
 '05',
 '1 POLISH AH AH AH',
 '100',
 '111',
 '2031C',
 '2124818001',
 '26351 EGRET COMBO',
 '26398 WHITE ONYX COMBO',
 '26451 VINTAGE INDIGO',
 '26451 VINTAGE INDIGO COMBO',
 '3 Designs',
 '3035C',
 '3D CLOUD/GREY',
 '3D CLOUD/PINK',
 '4 color photography',
 "5 O'CLOCK SOMEWHERE",
 '7066244635',
 '7527C',
 '7704C',
 '825 VERY BLACK',
 '832 NUDE BEIGE',
 '840 NATURAL BEIGE',
 '845 WARM BEIGE',
 '850 CREAMY BEIGE',
 '862 NATURAL TAN',
 '870 TOASTED ALMOND',
 'A GOOD DAY',
 'A HOT MINUTE',
 'A PERFECT TIN',
 'A SCARE IS BORN',
 'A WHOLE LOTTA WARMTH',
 "A Whole New Pearl'd",
 'A-DOUGH-ABLE',
 'ABALONE',
 'ABC123DEF',
 'ABSTRACT CAMO IRISH CREAM',
 'ABSTRACT FLORAL VINTAGE INDIGO',
 'ABSTRACT ORANGE',
 'ABYSS',
 'ACAI',
 'ACCENT COLOR- GRAY-LT.PUPRPLE-CORAL-MINT GREEN',
 'ACCENT COLOR- PINK-BLUE-GREEN-NEON -GREY',
 'ACCENT COLOR- PINK-BLUE-GREEN-NEON -GREY-PURPLE',
 'ACCENT COLOR- PINK-

In [10]:
replacements = {          
 ',': ' ',
 '/': ' ',
 '>':' ',
'<': ' '}


In [11]:
### Remove substrings like 'CHOCOLATE' if 'CHOCOLATE CHIP' is also present as a top result
def remove_substrings(string_list):
    str_df = pd.DataFrame(string_list, columns= ['Sentence'])
    str_df['Word Length'] = str_df.Sentence.apply(lambda x: len(x.split(' ')))
    str_df['Drop'] = ''
    str_df.sort_values('Word Length', inplace= True)
    str_df.reset_index(drop = True, inplace= True)
    for j in range(len(str_df)):
        str_1 = str_df.Sentence.loc[j]
        for i in range(len(str_df)-(j +1)):
            str_2 = str_df.Sentence.loc[i+j+1]
            if str_1 in str_2: 
                str_df.Drop.loc[j] = 'Yes'
                break
        if len(str_df) <2: 
            break
    return str_df[str_df.Drop != 'Yes'].Sentence.tolist()

In [12]:
#MAtch using 'token set ratio' with threshold
def color_match(item, color_list_org, cutoff = 90):
    #devowelized = [devowelize(i) for i in flavor_list_org]
    #derepeated = [remove_adjacent_repititions(i) for i in flavor_list_org]
    color_list = color_list_org #+ extras#+ devowelized
    #flavor_list += derepeated
    #abbrev_maps = dict(zip(devowelized+ derepeated, flavor_list_org + flavor_list_org))
    color_no_spaces = [ i for i in color_list_org if not '' in i]
    color_with_spaces = [ i for i in color_list_org if i not in color_no_spaces]
    abbrev_maps = {}
    for i in color_no_spaces:
        for j in color_with_spaces:
            if j.replace(' ','') == i:
                abbrev_maps[i] = j
    matched = process.extract(item, color_list, score_cutoff= cutoff, scorer = fuzz.token_set_ratio, processor=utils.default_process, limit= 7)
    color_shortlist = [i[0] for i in matched]
    matched = [i for i in matched if i[0] in color_shortlist]
    if len( color_shortlist) >0:
        max_score = matched[0][1]
        final_match = [ i for i in matched if i[1]== max_score]
        match_list = [i[0] for i in final_match]
        match_list = [abbrev_maps[i] if i in abbrev_maps.keys() else i for i in match_list]
        rematched = process.extract(item, match_list,  scorer = fuzz.partial_ratio, processor=utils.default_process, limit = 5)
        re_max_score = rematched[0][1]
        final_rematch = [ i for i in rematched if i[1]== re_max_score]
        match_list =  remove_substrings([i[0] for i in final_rematch])
        match_list_ordered =[]
        for word in item.split(' '):#Ordering scents according to thair appearance in the description text
            for color_token in match_list:
                if word in color_token:
                    match_list_ordered.append(color_token)
                    continue
        if len(match_list_ordered) >0:
            match_list = pd.Series(match_list_ordered).drop_duplicates().tolist()
        else:
            match_list = pd.Series(match_list).drop_duplicates().tolist()
    
        
        matched = '|'.join(list(pd.Series(match_list)))
    else: 
        matched = 'OTHER'
    return matched

### Match within DPT

In [13]:
colors_DPT = pd.DataFrame(pim_load.groupby('DPT_DSC').apply(lambda x: '|'.join(x.COLOR.unique())), columns= ['UNIQUE COLORS']).reset_index()
color_DPT_DSCs = colors_DPT[~colors_DPT['UNIQUE COLORS'].apply(lambda x: set(x.split('|')).issubset({'NONE','OTHER'}))].DPT_DSC.unique()
len(color_DPT_DSCs)


65

In [14]:
colors_COM = pd.DataFrame(pim_load.groupby('COM_CD').apply(lambda x: '|'.join(x.COLOR.unique())), columns= ['UNIQUE COLORS']).reset_index()
color_COM_CDs = colors_COM[~colors_COM['UNIQUE COLORS'].apply(lambda x: set(x.split('|')).issubset({'NONE','OTHER'}))].COM_CD.unique()
len(color_COM_CDs)

429

In [15]:
colors_COM.head()

,COM_CD,UNIQUE COLORS
0,0.00,NONE
1,1.00,NONE|GOLDEN
2,2.00,NONE|RED
3,3.00,NONE
4,4.00,NONE


In [16]:
#Exclude COMS without color
pim_load = pim_load[pim_load.COM_CD.isin(color_COM_CDs)]

In [17]:
#Concatenate multiple item description fields
pim_load['MATCHING_DESCRIPTION'] = \
pim_load.ITM_LVL_DSC.fillna('') + ' ' + pim_load.VND_ECOM_DSC +' '+  pim_load.TAG_DSC.fillna('')

In [18]:
color_list = [i.replace('_',' ') for i in pim_load['COLOR'].unique() if not i in  ['OTHER','NONE']]

In [19]:
pim_colorful = pim_load[~pim_load.COLOR.isin(['NONE'])]
pim_colorful.DPT_DSC.value_counts()[:10]

FURN/HOME DEC     269423
HOUSEWARES        137273
GARDEN-OUTDOOR    135935
DOMESTICS          53296
SEASONAL           48092
TOYS               39031
CARDS              39017
SCHOOL/OFFICE      37310
HOME IMPRVMNT      25726
MENS WEAR          24327
Name: DPT_DSC, dtype: int64

In [20]:
import time
from datetime import date

In [21]:
dte = date.today().strftime('%m%d%y')
excel_file = path_atb + " COLOR_DPT_DSC_"+dte + '.xlsx'
writer = pd.ExcelWriter(excel_file)

In [22]:
colors_DPT['DPT_DSC'] = colors_DPT.DPT_DSC.fillna('MISSING') 
df_dpt = pd.DataFrame() 
l_sumy = []
display_cols = ['GTIN_NO','PMY_DPT_DSC','REC_DPT_DSC','DPT_DSC','COM_DSC','SUBCOM_DSC','VND_ECOM_DSC','ITM_LVL_DSC','COLOR_CURRENT','EXTRACTED_COLOR']
start_time_all = time.time() 
print(f'The length of colors_DPT.DPT_DSC.unique() is {len(colors_DPT.DPT_DSC.unique())}')


The length of colors_DPT.DPT_DSC.unique() is 95


In [23]:
seq = 0 
#for dpt in colors_DPT.DPT_DSC.unique():#, 'HOUSEWARES', PERSONAL CARE','PET','BEAUTY']:# Choose from scents_DPT.DPT_DSC.unique():
for dpt in ['HOUSEWARES']:
    seq += 1 

    pim_dpt = pim_load[pim_load.DPT_DSC == dpt]    
    len_pim_load = len(pim_load)
    len_pim_dpt = len(pim_dpt)
   
    start_time = time.time()         
    print(f'{seq}. {dpt} with length= {len_pim_dpt}' )
    

    if len_pim_dpt ==0:
       print(f' There is no observations in pim_load for  dpt = {dpt} in colors_DPT. Skip matching process' ) 
       continue    
        
    if '<OBSOLETE>' in dpt :
       print(f' There dpt is OBSOLETE. Skip matching process' )
       continue
        
    pim_dpt.rename(columns={'COLOR':'COLOR_CURRENT'}, inplace= True)
    pim_dpt['EXTRACTED_COLOR']  =  \
                   pim_dpt.MATCHING_DESCRIPTION.fillna('').str.upper().replace(replacements).apply(lambda x:color_match(x, color_list_org= color_list))# + [i.replace(" ","") for  i in  scent_list]))   
   

    len_flv_oth = len(pim_dpt[pim_dpt['EXTRACTED_COLOR'] == 'OTHER'])
    len_flv_not_oth = len(pim_dpt[pim_dpt['EXTRACTED_COLOR'] != 'OTHER'])
           
    end_time = time.time() 
    elapsed =  end_time -  start_time
    hh, mm, ss = f_get_hh_mm_ss(elapsed)
    elap_time = f'{hh}hh:{mm}mm:{ss}ss' 
    l_sumy.append( [dpt, len_pim_dpt,  round(len_pim_dpt/len_pim_load, 4), len_flv_not_oth,round(len_flv_not_oth/len_pim_dpt, 4), 
          len_flv_oth,  round(len_flv_oth/len_pim_dpt, 4), elap_time])         

    dpt = re.sub("/",'_',dpt)
    pim_dpt[display_cols].to_excel(writer, sheet_name= dpt, index = None)        
    df_dpt = pd.concat ([df_dpt, pim_dpt])  

1. HOUSEWARES with length= 361910


In [24]:
df_sumy = pd.DataFrame(data = l_sumy, columns = ['dpt', 'dpt_lng', 'dpt_ratio', 'non_oth_cnt', 'not_oth_ratio', 'oth_cnt', 'oth_cnt_ratio', 'elapse time'])
df_sumy.to_excel(writer, sheet_name= 'sumy', index = None) 
#pim_load.to_excel(writer, sheet_name= 'pim_load') # Too large
#df_dpt.to_excel(writer, sheet_name= 'df_dpt')  # Too large
writer.close()
end_time_all = time.time() 
elapsed =  end_time_all-  start_time_all
hh, mm, ss = f_get_hh_mm_ss(elapsed)
elap_time = f'{hh}hh:{mm}mm:{ss}ss'
print(f' there are {len(color_COM_CDs)} colors in color_COM_CDs')
print(f'The elapsed_time to run  the whole dpt is {elap_time}' )

 there are 429 colors in color_COM_CDs
The elapsed_time to run  the whole dpt is 1hh:10mm:49ss


In [25]:
stop

NameError: name 'stop' is not defined

In [ ]:
writer.save()

In [ ]:
df_sumy = pd.DataFrame(data = l_sumy, columns = ['dpt', 'dpt_lng', 'dpt_ratio', 'non_oth_cnt', 'not_oth_ratio', 'oth_cnt', 'oth_cnt_ratio', 'elapse time'])
df_sumy.to_excel(path_atb + 'color_match_dpt_dsc_sumy_111723.xlsx', index = None)

elapsed =  end_time-  start_time_all
hh, mm, ss = f_get_hh_mm_ss(elapsed)
elap_time = f'{hh}hh:{mm}mm:{ss}ss'
print(f' there are {len(color_COM_CDs)} colors in color_COM_CDs')
print(f'The elapsed_time to run  the whole dpt is {elap_time}' )

In [ ]:
pim_load.DPT_DSC.value_counts(dropna =  False)

In [ ]:
 colors_DPT.DPT_DSC.unique()

In [ ]:
pim_load.MATCHING_DESCRIPTION.isnull().sum()

In [ ]:
print ( f'color_list {color_list}')

In [ ]:
len(color_list)

In [ ]:
colors_DPT.head()